In [1]:
!pip install -q mediapipe

In [3]:
!curl -o pose_landmarker_heavy.task https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 29.2M    0 14990    0     0  40145      0  0:12:43 --:--:--  0:12:43 40513
  9 29.2M    9 2800k    0     0  2108k      0  0:00:14  0:00:01  0:00:13 2113k
 15 29.2M   15 4720k    0     0  2025k      0  0:00:14  0:00:02  0:00:12 2028k
 23 29.2M   23 7088k    0     0  2128k      0  0:00:14  0:00:03  0:00:11 2130k
 35 29.2M   35 10.4M    0     0  2245k      0  0:00:13  0:00:04  0:00:09 2246k
 41 29.2M   41 12.2M    0     0  2203k      0  0:00:13  0:00:05  0:00:08 2356k
 47 29.2M   47 14.0M    0     0  2264k      0  0:00:13  0:00:06  0:00:07 2305k
 55 29.2M   55 16.2M    0     0  2265k      0  0:00:13  0:00:07  0:00:06 2377k
 66 29.2M   66 19.3M    0     0  2294k      0  0:00:13  0:00:08  0:00:05 2400k
 71 29.2M   71 20.8M    0     0  2242k      0  0:00

In [5]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import json
import csv
import os

In [7]:
def draw_landmarks_on_image(rgb_image, detection_result):
    pose_landmarks_list = detection_result.pose_landmarks
    annotated_image = np.copy(rgb_image)

    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]
        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        pose_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            pose_landmarks_proto,
            solutions.pose.POSE_CONNECTIONS,
            solutions.drawing_styles.get_default_pose_landmarks_style())
    return annotated_image

In [11]:
def extract_landmarks(detection_result):
    landmarks = []
    for pose_landmarks in detection_result.pose_landmarks:
        for landmark in pose_landmarks:
            landmarks.append({
                'x': landmark.x,
                'y': landmark.y,
                'z': landmark.z,
                'visibility': landmark.visibility
            })
    return landmarks

In [18]:
base_options = python.BaseOptions(model_asset_path='pose_landmarker_heavy.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# Ruta base donde están las carpetas de datos
base_path = 'datos_Mateo'

# Iterar sobre cada categoría de movimiento en la carpeta base
for category in os.listdir(base_path):
    category_path = os.path.join(base_path, category)
    
    # Revisar que sea una carpeta
    if not os.path.isdir(category_path):
        continue
    
    # Crear carpeta de salida para JSON y CSV de la categoría si no existe
    output_json_path = os.path.join(category_path, 'json')
    output_csv_path = os.path.join(category_path, 'csv')
    os.makedirs(output_json_path, exist_ok=True)
    os.makedirs(output_csv_path, exist_ok=True)
    
    # Procesar cada video en la categoría
    for video_file in os.listdir(category_path):
        if not video_file.endswith('.mp4'):
            continue
        
        video_path = os.path.join(category_path, video_file)
        print(video_path)
        cap = cv2.VideoCapture(video_path)
        all_landmarks = []

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
            detection_result = detector.detect(mp_image)

            landmarks = extract_landmarks(detection_result)
            all_landmarks.append(landmarks)

        cap.release()

        # Guardar los datos en formato JSON
        json_output_path = os.path.join(output_json_path, f"{video_file.split('.')[0]}.json")
        with open(json_output_path, 'w') as json_file:
            json.dump(all_landmarks, json_file, indent=4)

        # Guardar los datos en formato CSV
        csv_output_path = os.path.join(output_csv_path, f"{video_file.split('.')[0]}.csv")
        csv_columns = ['frame', 'landmark_index', 'x', 'y', 'z', 'visibility']
        with open(csv_output_path, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for frame_index, frame_landmarks in enumerate(all_landmarks):
                for landmark_index, landmark in enumerate(frame_landmarks):
                    row = {
                        'frame': frame_index,
                        'landmark_index': landmark_index,
                        'x': landmark['x'],
                        'y': landmark['y'],
                        'z': landmark['z'],
                        'visibility': landmark['visibility']
                    }
                    writer.writerow(row)


datos_Mateo\caminar hacia atras\VID-20241028-WA0019.mp4
datos_Mateo\caminar hacia atras\VID-20241028-WA0020.mp4
datos_Mateo\caminar hacia atras\VID-20241028-WA0021.mp4
datos_Mateo\caminar hacia atras\VID-20241028-WA0022.mp4
datos_Mateo\caminar hacia delante\VID-20241028-WA0013.mp4
datos_Mateo\caminar hacia delante\VID-20241028-WA0014.mp4
datos_Mateo\caminar hacia delante\VID-20241028-WA0015.mp4
datos_Mateo\caminar hacia delante\VID-20241028-WA0016.mp4
datos_Mateo\caminar hacia delante\VID-20241028-WA0017.mp4
datos_Mateo\caminar hacia delante\VID-20241028-WA0018.mp4
datos_Mateo\pararse\VID-20241028-WA0008.mp4
datos_Mateo\pararse\VID-20241028-WA0009.mp4
datos_Mateo\pararse\VID-20241028-WA0010.mp4
datos_Mateo\pararse\VID-20241028-WA0011.mp4
datos_Mateo\pararse\VID-20241028-WA0012.mp4
datos_Mateo\Sentarse\1.mp4
datos_Mateo\Sentarse\2.mp4
datos_Mateo\Sentarse\3.mp4
datos_Mateo\Sentarse\4.mp4
datos_Mateo\Sentarse\5.mp4
